# Table basics

OOMMF saves the scalar data of micromagnetic simulations in `.odt` files, whereas mumax$^{3}$ saves it in `.txt` files. `ubermagtable` is a convenience tool which provides functions for reading, manipulating, and visualising data from those files.

As an example, we are going to use a simple macrospin example:

In [2]:
import oommfc as mc
import discretisedfield as df
import micromagneticmodel as mm

# Define a macrospin mesh (i.e. one discretisation cell).
p1 = (0, 0, 0)  # first point of the mesh domain (m)
p2 = (1e-9, 1e-9, 1e-9)  # second point of the mesh domain (m)
n = (1, 1, 1)  # discretisation cell size (m)

Ms = 8e6  # magnetisation saturation (A/m)
H = (0, 0, 2e6)  # external magnetic field (A/m)
gamma0 = 2.211e5  # gyromagnetic ratio (m/As)
alpha = 0.1  # Gilbert damping

region = df.Region(p1=p1, p2=p2)
mesh = df.Mesh(region=region, n=n)

system = mm.System(name='macrospin')
system.energy = mm.Zeeman(H=H)
system.dynamics = mm.Precession(gamma0=gamma0) + mm.Damping(alpha=alpha)
system.m = df.Field(mesh, dim=3, value=(1, 0, 0), norm=Ms)

td = mc.TimeDriver()
td.drive(system, t=0.1e-9, n=200)

Running OOMMF (ExeOOMMFRunner) [2020/07/02 15:14]... (2.9 s)


Table data is now loaded into `pandas.DataFrame` and it can be accessed via `data` attribute.

In [4]:
system.table.data

,E,E_calc_count,max_dm/dt,dE/dt,delta_E,E_zeeman,iteration,stage_iteration,stage,mx,my,mz,last_time_step,t
0,-4.400762e-22,37.0,25204.415522,-8.798712e-10,-3.269612e-22,-4.400762e-22,6.0,6.0,0.0,0.975901,0.217115,0.021888,3.715017e-13,5.000000e-13
1,-8.797309e-22,44.0,25186.311578,-8.786077e-10,-4.396547e-22,-8.797309e-22,8.0,1.0,1.0,0.904810,0.423562,0.043754,5.000000e-13,1.000000e-12
2,-1.318544e-21,51.0,25156.186455,-8.765071e-10,-4.388134e-22,-1.318544e-21,10.0,1.0,2.0,0.790286,0.609218,0.065579,5.000000e-13,1.500000e-12
3,-1.756100e-21,58.0,25114.112032,-8.735776e-10,-4.375555e-22,-1.756100e-21,12.0,1.0,3.0,0.638055,0.765021,0.087341,5.000000e-13,2.000000e-12
4,-2.191985e-21,65.0,25060.188355,-8.698302e-10,-4.358857e-22,-2.191985e-21,14.0,1.0,4.0,0.455710,0.883427,0.109020,5.000000e-13,2.500000e-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,-2.009865e-20,1402.0,690.438568,-6.602614e-13,-3.374608e-25,-2.009865e-20,396.0,1.0,195.0,0.013011,-0.024099,0.999625,5.000000e-13,9.800000e-11
196,-2.009897e-20,1409.0,675.493807,-6.319876e-13,-3.230101e-25,-2.009897e-20,398.0,1.0,196.0,0.017545,-0.020251,0.999641,5.000000e-13,9.850000e-11
197,-2.009928e-20,1416.0,660.872303,-6.049242e-13,-3.091780e-25,-2.009928e-20,400.0,1.0,197.0,0.021059,-0.015612,0.999656,5.000000e-13,9.900000e-11
198,-2.009958e-20,1423.0,646.567078,-5.790193e-13,-2.959381e-25,-2.009958e-20,402.0,1.0,198.0,0.023428,-0.010435,0.999671,5.000000e-13,9.950000e-11


Apart from the data, units for individual columns are stored in `units` attribute.

In [5]:
system.table.units

{'E': 'J',
 'E_calc_count': '',
 'max_dm/dt': 'deg/ns',
 'dE/dt': 'J/s',
 'delta_E': 'J',
 'E_zeeman': 'J',
 'iteration': '',
 'stage_iteration': '',
 'stage': '',
 'mx': '',
 'my': '',
 'mz': '',
 'last_time_step': 's',
 't': 's'}

This returns a dictionary, whose keys are column names and its values are the units. If table data was obtained by using time driver, table data store different values which are all a function of time. Column names of all time-dependent data can be obtained using `data_columns`:

In [7]:
system.table.data_columns

['E',
 'E_calc_count',
 'max_dm/dt',
 'dE/dt',
 'delta_E',
 'E_zeeman',
 'iteration',
 'stage_iteration',
 'stage',
 'mx',
 'my',
 'mz',
 'last_time_step']

Similarly, column name storing time data is:

In [8]:
system.table.time_column

't'

The total length of time interval over which data was recorded is:

In [9]:
system.table.length

1e-10